In [1]:
!pip install schedule
!pip install snowflake-connector-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )
# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
output_excel_file = 'transformed_data_output.xlsx'  # Use the same file for all recipients

emails_df = pd.read_csv("C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv")

def transform(query):
    # Execute Snowflake SQL Queries
    # Connecting to Snowflake
    conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )
    cur = conn.cursor()
    cur.execute(f"USE {database}.{schema}")

    result_df = pd.read_sql_query(query, conn)

    # Identify columns of object type
    object_columns = result_df.select_dtypes(include='object').columns

    # Convert object columns to datetime
    for col in object_columns:
        try:
            converted_values = pd.to_datetime(result_df[col], errors='coerce')
            if converted_values.notna().any():
                result_df[col] = converted_values
        except ValueError:
            # Handle any specific errors that might occur during conversion
            pass
    return result_df

def send_email(name, email):
    # Create Excel attachment
    with open(output_excel_file, 'rb') as file:
        excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
        excel_attachment.add_header('Content-Disposition', 'attachment', filename='transformed_data_output.xlsx')

    # Create email message
    email_message = MIMEMultipart()
    email_message['From'] = sender_email
    email_message['To'] = email
    email_message['Subject'] = 'Transformed Data'
    email_message.attach(excel_attachment)

    # Customize email body
    email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
    email_message.attach(MIMEText(email_body, 'plain'))

    # Send email using SMTP
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, email, email_message.as_string())

def fetch_transform_send():
    # Define queries for each sheet with specific columns
    queries = [
        """
        SELECT ID, NAME
        FROM eggozdb.maplemonk.my_sql_product_product
        """,
        """
        SELECT ID, CODE
        FROM eggozdb.maplemonk.my_sql_retailer_retailer
        """,
        """
        SELECT DATE, SKU_ORDER_PRICE_AMOUNT
        FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
        """
    ]

    # Open the Excel file in 'w' (write) mode using openpyxl engine
    with pd.ExcelWriter(output_excel_file, engine='openpyxl', mode='w') as writer:
        # Add a default empty sheet to avoid the 'IndexError'
        pd.DataFrame().to_excel(writer, sheet_name='DefaultSheet', index=False)

        # Iterate over queries and send emails
        for sheet_index, query in enumerate(queries, start=1):
            result_df = transform(query)

            # Save DataFrame to the current sheet
            excel_sheet_name = f'Sheet{sheet_index}'

            # Check if the sheet already exists
            if excel_sheet_name in writer.sheets:
                # If the sheet exists, you can choose to overwrite or use a different name
                excel_sheet_name += "_new"  # Use a different name (you can customize this logic)

            result_df.to_excel(writer, sheet_name=excel_sheet_name, index=False)

            # Send customized emails to each recipient for the current sheet
            for index, row in emails_df.iterrows():
                send_email(row['name'], row['email'])

        # Close the connection to Snowflake
    conn.close()

# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)

Every 1 minute do fetch_transform_send() (last run: [never], next run: 2023-12-24 19:47:10)

In [11]:
while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\User\AppData\Local\Temp\ipykernel_6316\2781615253.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_6316\2781615253.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_6316\2781615253.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
Exception ignored in: <function SnowflakeConnection.__del__ at 0

In [7]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time
import os

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Connecting to Snowflake
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
output_excel_file = 'transformed_data_output.xlsx'  # Use the same file for all recipients


# Check if the Excel file already exists
if os.path.isfile(output_excel_file):
    # Try to open the Excel file in 'a' (append) mode using openpyxl engine
    with pd.ExcelWriter(output_excel_file, engine='openpyxl', mode='a') as writer:
        pass  # Do nothing, just need to check if the file exists
else:
    # If the file doesn't exist, create it with a dummy sheet
    with pd.ExcelWriter(output_excel_file, engine='openpyxl', mode='w') as writer:
        # Create a dummy sheet to avoid the "At least one sheet must be visible" error
        dummy_df = pd.DataFrame()
        dummy_df.to_excel(writer, sheet_name='dummy_sheet', index=False)

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')
print(emails_df.columns)
# Open the Excel file in 'a' (append) mode using openpyxl engine
with pd.ExcelWriter(output_excel_file, engine='openpyxl', mode='a') as writer:

    def transform(query):
        # Execute Snowflake SQL Queries
        # Connecting to Snowflake
        conn = snowflake.connector.connect(
            user=username,
            password=password,
            account=account,
            warehouse=warehouse,
            database=database,
            schema=schema
        )
        cur = conn.cursor()
        cur.execute(f"USE {database}.{schema}")

        result_df = pd.read_sql_query(query, conn)

        # Identify date columns and convert them to date without timezone
        date_columns = [col for col in result_df.columns if result_df[col].dtype == 'datetime64[ns, UTC]']

        for date_column in date_columns:
            # Check if the date column exists before attempting to convert
            if date_column in result_df.columns:
                result_df[date_column] = result_df[date_column].dt.date

        return result_df


    def send_email(name, email):
        # Create Excel attachment
        with open(output_excel_file, 'rb') as file:
            excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
            excel_attachment.add_header('Content-Disposition', 'attachment', filename='transformed_data_output.xlsx')

        # Create email message
        email_message = MIMEMultipart()
        email_message['From'] = sender_email
        email_message['To'] = email
        email_message['Subject'] = 'Transformed Data'
        email_message.attach(excel_attachment)

        # Customize email body
        email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
        email_message.attach(MIMEText(email_body, 'plain'))

        # Send email using SMTP
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, app_password)
            server.sendmail(sender_email, email, email_message.as_string())

    def fetch_transform_send():
        # Define queries for each sheet with specific columns
        queries = [
            """
            SELECT QUANTITY, SKU_ORDER_PRICE_AMOUNT
            FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
            """,
            """
            SELECT ID, NAME
            FROM eggozdb.maplemonk.my_sql_product_product
            """,
            """
            SELECT ID, CODE
            FROM eggozdb.maplemonk.my_sql_retailer_retailer
            """
        ]

        # Iterate over queries and send emails
        for sheet_index, (query, sheet_name_suffix) in enumerate(zip(queries, ["Sheet1_akt", "Sheet2_abhi", "Sheet3_eggoz"]), start=1):
            result_df = transform(query)

            # Save DataFrame to the current sheet
            excel_sheet_name = f'{sheet_name_suffix}_new{sheet_index}'

            # Check if the sheet already exists
            if excel_sheet_name in writer.sheets:
                # If the sheet exists, you can choose to overwrite or use a different name
                excel_sheet_name += "_new"  # Use a different name (you can customize this logic)

            result_df.to_excel(writer, sheet_name=excel_sheet_name, index=False)

            # Send customized emails to each recipient for the current sheet
            for index, row in emails_df.iterrows():
                send_email(row['name'], row['email'])

        # Close the connection to Snowflake
        conn.close()

# Schedule the message to be sent every hour
schedule.every(1).minute.do(fetch_transform_send)


BadZipFile: File is not a zip file

In [5]:
while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\User\AppData\Local\Temp\ipykernel_7604\2684987354.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_7604\2684987354.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_7604\2684987354.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)


KeyboardInterrupt: 

In [9]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time
from datetime import datetime, timezone

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Connecting to Snowflake
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
excel_file_path = 'transformed_data.xlsx'  # Use the same file for all recipients

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')
print(emails_df.columns)
def transform(query):
    # Execute Snowflake SQL Queries
    # Connecting to Snowflake
    conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )
    cur = conn.cursor()
    cur.execute(f"USE {database}.{schema}")


    result_df = pd.read_sql_query(query, conn)

    # Identify columns of object type
    object_columns = result_df.select_dtypes(include='object').columns

    # Convert object columns to datetime
    for col in object_columns:
        try:
            converted_values = pd.to_datetime(result_df[col], errors='coerce')
            if converted_values.notna().any():
                result_df[col] = converted_values
        except ValueError:
            # Handle any specific errors that might occur during conversion
            pass
    return result_df

def send_email(name, email):
    # Create Excel attachment
    with open(excel_file_path, 'rb') as file:
        excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
        excel_attachment.add_header('Content-Disposition', 'attachment', filename='transformed_data.xlsx')

    # Create email message
    email_message = MIMEMultipart()
    email_message['From'] = sender_email
    email_message['To'] = email
    email_message['Subject'] = 'Transformed Data'
    email_message.attach(excel_attachment)

    # Customize email body
    email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
    email_message.attach(MIMEText(email_body, 'plain'))

    # Send email using SMTP
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, email, email_message.as_string())

def fetch_transform_send():
    # Define queries for each sheet with specific columns
  
    # Define queries for each sheet with specific columns
    queries = [
        """
        SELECT ID, NAME
        FROM eggozdb.maplemonk.my_sql_product_product
        """,
        """
        SELECT ID, CODE
        FROM eggozdb.maplemonk.my_sql_retailer_retailer
        """,
        """
        SELECT DATE, SKU_ORDER_PRICE_AMOUNT
        FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
        """
    ]
    
    # Iterate over queries and send emails
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter', mode='w') as writer:
        for sheet_index, query in enumerate(queries, start=1):
            result_df = transform(query)

            # Save DataFrame to an Excel file on a different sheet
            excel_sheet_name = f'Sheet{sheet_index}'
            result_df.to_excel(writer, sheet_name=excel_sheet_name, index=False)

            # Send customized emails to each recipient for the current sheet
            for index, row in emails_df.iterrows():
                send_email(row['name'], row['email'])
                if index == len(emails_df) - 1:
                    break

    conn.close()

# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)

Index(['name', 'email'], dtype='object')


Every 1 minute do fetch_transform_send() (last run: [never], next run: 2023-12-25 11:39:41)

In [10]:
while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\User\AppData\Local\Temp\ipykernel_7604\2151509413.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
Exception ignored in: <function ZipFile.__del__ at 0x00000278E8046440>
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\zipfile.py", line 1808, in __del__
    self.close()
  File "c:\Program Files\Python310\lib\zipfile.py", line 1825, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [3]:
!pip install xlsxwriter


  Using cached XlsxWriter-3.1.9-py3-none-any.whl (154 kB)


ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'C:\\Python311\\Scripts\\vba_extract.py'
Consider using the `--user` option or check the permissions.



In [3]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time
import os

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'

# Set the output directory in your local environment
output_dir = 'C:\\Users\\User\\Downloads\\Eggoz\\Customer_Segmentation'

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Set the Excel file name
excel_file_name = 'transformed_data_output.xlsx'

# Construct the full path to the Excel file
excel_file_path = os.path.join(output_dir, excel_file_name)

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')
print(emails_df.columns)

def transform(query):
    # Execute Snowflake SQL Queries
    # Connecting to Snowflake
    conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )
    cur = conn.cursor()
    cur.execute(f"USE {database}.{schema}")

    result_df = pd.read_sql_query(query, conn)

    # Identify columns of object type
    object_columns = result_df.select_dtypes(include='object').columns

    # Convert object columns to datetime
    for col in object_columns:
        try:
            converted_values = pd.to_datetime(result_df[col], errors='coerce')
            if converted_values.notna().any():
                result_df[col] = converted_values
        except ValueError:
            # Handle any specific errors that might occur during conversion
            pass
    return result_df

def send_email(name, email):
    # Create Excel attachment
    with open(excel_file_path, 'rb') as file:
        excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
        excel_attachment.add_header('Content-Disposition', 'attachment', filename='transformed_data_output.xlsx')

    # Create email message
    email_message = MIMEMultipart()
    email_message['From'] = sender_email
    email_message['To'] = email
    email_message['Subject'] = 'Transformed Data'
    email_message.attach(excel_attachment)

    # Customize email body
    email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
    email_message.attach(MIMEText(email_body, 'plain'))

    # Send email using SMTP
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, email, email_message.as_string())

def fetch_transform_send():
    # Define queries for each sheet with specific columns
    queries = [
        """
        SELECT ID, NAME
        FROM eggozdb.maplemonk.my_sql_product_product
        """,
        """
        SELECT ID, CODE
        FROM eggozdb.maplemonk.my_sql_retailer_retailer
        """,
        """
        SELECT DATE, SKU_ORDER_PRICE_AMOUNT
        FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
        """
    ]

    # Use a single ExcelWriter for all three queries
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter', mode='a') as writer:
        for sheet_index, query in enumerate(queries, start=1):
            result_df = transform(query)

            # Save DataFrame to an Excel file on a different sheet
            excel_sheet_name = f'Sheet{sheet_index}'
            result_df.to_excel(writer, sheet_name=excel_sheet_name, index=False)

        # Send customized emails to each recipient for the Excel file
        for index, row in emails_df.iterrows():
            send_email(row['name'], row['email'])

# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)


Index(['name', 'email'], dtype='object')


Every 1 minute do fetch_transform_send() (last run: [never], next run: 2023-12-25 12:37:12)

In [4]:
while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\User\AppData\Local\Temp\ipykernel_12740\3989504763.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_12740\3989504763.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_12740\3989504763.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)


ValueError: Append mode is not supported with xlsxwriter!